# Adriann Liceralde
# Combine CSV - Binary NEW

## Select Directory
1. Import all necessary libraries.
2. Set the directory to the folder that contains the target dataset.
3. Select all the csv files within the folder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

In [ ]:
#user = 'Adriann Liceralde'
user      = 'adria'
subfolder = 'V 2.6.1 Roof-Primary'
#subfolder = 'V 2.6.2 Roof-Secondary'
os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\Repo\\WirelessSensorNetwork\\Data\\'+ str(subfolder))
#os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\'+str(subfolder))
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

## Process Binary Data
1. Establish the data type class.
2. Send information from each file over to a main numpy array
3. Turn numpy array into pandas datafrane

In [ ]:
first  = True
mainNP = np.array([])
dt     = np.dtype( [('unixtime','>u4'), ('ws','u1'), ('wd','>u2'), ('co2','>u2'), ('temp','u1')] )

for each in all_filenames:
    filesize   = int(os.path.getsize(each))
    headersize = 0
    lines      = (filesize-headersize)//10
    print('File :' , each, '\n', 'Bytes:' , filesize, '\n', 'Lines:' , lines)

    fileNP = np.fromfile(each, dtype = dt)
    
    if (first == True):
        mainNP = fileNP
        first = False
    elif (first == False):
        mainNP = np.concatenate( (mainNP, fileNP))
        
mainDF = pd.DataFrame(mainNP)
mainDF = mainDF.assign(date = pd.to_datetime(mainDF.unixtime, unit = 's').dt.strftime('%m/%d/%Y %H:%M'))
mainDF.ws = (mainDF.ws*2.25/3/2.237).round(2)
mainDF = mainDF[['unixtime', 'date', 'ws', 'wd', 'co2', 'temp']]

In [ ]:
mainDF

## Export to Single CSV
1. Select the name for the finalized csv file
2. Export the pandas dataframe to a csv file into a different folder directory
3. Check to see if all the timestamps in the dataset have been accounted for.

In [ ]:
node   = subfolder[12:16]
target = subfolder[0:6]+str(0)+(' Roof')
os.chdir('C:\\Users\\'+str(user)+'\\Desktop\\Repo\\WirelessSensorNetwork\\Data\\'+ str(target))

name = 'C-'+ str(all_filenames[0])[0:-8] + str(node)+ '.csv'
mainDF.to_csv(name, index=False, encoding='utf-8-sig')

In [ ]:
#Check if all timestamps are accounted for
first = mainNP[0][0]
last  = mainNP[-1][0]
span = np.arange(first, last+3, 3)
unix = mainDF.unixtime.to_numpy()
summation = sum(unix-span)
print(summation)